In [1]:
import functools

import jax
import numpy as np
import scanpy as sc
import cfp.preprocessing as cfpp
from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast

In [2]:
ood_split=5

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{ood_split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{ood_split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{ood_split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)
adata_pred_test = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/biolord_output_test_{ood_split}.h5ad")
adata_pred_ood = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/biolord_output_ood_{ood_split}.h5ad")

In [5]:
adata_ref_ood = adata_ood[adata_ood.obs["condition"].str.contains('Vehicle')].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

adata_ref_test = adata_test[adata_test.obs["condition"].str.contains('Vehicle')].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

In [6]:

cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test)
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].X
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

In [7]:

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [8]:

test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}

ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

#test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
#test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

#deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
#deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [12]:
#test_metrics_encoded = jax.tree_util.tree_map(
#    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
#)
#mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

#test_metrics_decoded = jax.tree_util.tree_map(
#   compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
#)
#mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded

{'A549_Alvespimycin_(17-DMAG)_HCl_10.0': {'r_squared': 0.9949608445167542,
  'sinkhorn_div_1': 1.0172791481018066,
  'sinkhorn_div_10': 0.23923778533935547,
  'sinkhorn_div_100': 0.09623146057128906,
  'e_distance': 0.16112688231749095,
  'mmd': 0.042976256},
 'A549_Alvespimycin_(17-DMAG)_HCl_100.0': {'r_squared': 0.9386213421821594,
  'sinkhorn_div_1': 2.0647430419921875,
  'sinkhorn_div_10': 1.2535696029663086,
  'sinkhorn_div_100': 1.1110153198242188,
  'e_distance': 2.191717206809276,
  'mmd': 0.08915347},
 'A549_Belinostat_(PXD101)_10.0': {'r_squared': 0.9897606372833252,
  'sinkhorn_div_1': 1.8515558242797852,
  'sinkhorn_div_10': 0.5192680358886719,
  'sinkhorn_div_100': 0.20453453063964844,
  'e_distance': 0.33828416164315644,
  'mmd': 0.07562771},
 'A549_Belinostat_(PXD101)_100.0': {'r_squared': 0.9825670123100281,
  'sinkhorn_div_1': 1.775285243988037,
  'sinkhorn_div_10': 0.5796623229980469,
  'sinkhorn_div_100': 0.30599212646484375,
  'e_distance': 0.5503128759630722,
  'mm

In [14]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.7169944593705326,
 'encoded_ood_sinkhorn_div_1': 2.689353562263121,
 'encoded_ood_sinkhorn_div_10': 1.6295737731887634,
 'encoded_ood_sinkhorn_div_100': 1.30375066722732,
 'encoded_ood_e_distance': 2.5363255016650816,
 'encoded_ood_mmd': 0.10717861637114043}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8606915150780275,
 'decoded_ood_sinkhorn_div_1': 48.99501061726765,
 'decoded_ood_sinkhorn_div_10': 24.72928426351892,
 'decoded_ood_sinkhorn_div_100': 5.254168820668416,
 'decoded_ood_e_distance': 9.760340693599952,
 'decoded_ood_mmd': 0.0507709109729313}

In [16]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/results/biolord"

In [19]:
import os
import pandas as pd
split=ood_split
pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
#pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
#pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
#pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
